### Development of Code for Machine Learning Approach to Feedback Loop

Goal: create neural net which can be fine-tuned every 24 hours based on data received that day
- may need initial data to start this

Targeting optimal watering times, so this is a regression issue - perhaps can frame it as an optimal "pace"
- if we treat the whole 24 hours as a single block, there is some optimal pace of watering, aka x amount of L over 24 hours
- but we should probably break up the 24 hours into 4 or 6 hour blocks, which have different sunlight and temperatures on average, so there are optimal paces for each of those blocks (influenced by the pace chosen for each other block). That makes it a multi-output regression problem
- we'll start by treating each zone as a seperate entity

Our goal is to stabilize soil moisture and pH at desireable levels
- can change if we find other variables we can influence or target but I think those are two that make sense
- We can cut out pH if it makes it unecessarily complicated, and should put more weight on soil moisture anyway
- We must come up with a loss function that involves soil moisture and pH in some way

Actually:
- we are learning soil moisture and ph level based on environmental conditions
- from there we can then apply it to create a schedule

In [94]:
import pandas as pd
import random
from datetime import datetime
import json

#### Data Processing

In [14]:
# How do we get and store data that we watered it and when?

watering_df = pd.read_csv("../database/watering.csv")
watering_df.head()

,time,zone,amount (L)
0,Tue--Dec----5--00:00:00,A,389.485765
1,Tue--Dec----5--00:00:00,B,293.624334
2,Tue--Dec----5--01:00:00,A,66.365255
3,Tue--Dec----5--01:00:00,B,386.107168
4,Tue--Dec----5--02:00:00,A,284.108067


In [59]:
# Weather JSON data

weather_df = pd.read_csv("../database/weather.csv")
weather_df = pd.concat([weather_df, pd.DataFrame({"zone":["all"]*len(weather_df)})], axis=1)
weather_df.head()

# how do we deal with time?
# scan for outliers?

,time,temperature,weather,zone
0,Tue--Dec----5--00:00:00,60.335256,heavy intensity rain,all
1,Tue--Dec----5--00:30:00,64.656459,broken clouds: 51-84%,all
2,Tue--Dec----5--01:00:00,56.923054,fog,all
3,Tue--Dec----5--01:30:00,41.564889,thunderstorm with light rain,all
4,Tue--Dec----5--02:00:00,40.777651,thunderstorm with light rain,all


In [15]:
# Sensor JSON data

sensor_df = pd.read_csv("../database/sensors.csv")
sensor_df.head()

# how do we deal with time?
# scan for outliers or errors?

,time,zone,moisture,humidity,temperature,ir,vis,uv,ph
0,Tue--Dec----5--00:00:00,A,77.139656,6.375502,59.271949,26.015057,92.725291,31.973472,7.615026
1,Tue--Dec----5--00:00:00,B,48.629872,69.802679,56.391598,96.123475,33.996783,94.558314,6.656450
2,Tue--Dec----5--00:15:00,A,25.607557,30.231270,77.365120,93.836455,30.876055,82.859301,7.598607
3,Tue--Dec----5--00:15:00,B,10.069499,16.320478,76.695698,81.613220,13.091263,69.335290,7.589150
4,Tue--Dec----5--00:30:00,A,75.512344,21.538457,70.814017,62.383182,44.635027,71.770407,5.771829


In [72]:
zones = list(pd.unique(sensor_df["zone"]))
x_dfs = {}
y_dfs = {}

for zone in zones:
    # Join all based on time:
    watering_df_zone = watering_df[watering_df["zone"] == zone]
    sensor_df_zone = sensor_df[sensor_df["zone"] == zone]
    joined_df = pd.concat([sensor_df_zone, pd.DataFrame({"amount (L)":[], "weather": []})], ignore_index=True)
    for i in range(len(joined_df)):

        time_str = joined_df["time"].iat[i]

        if (len(watering_df_zone[watering_df_zone["time"] == time_str]) == 1):
            joined_df["amount (L)"].iat[i] = watering_df_zone[watering_df_zone["time"] == time_str]["amount (L)"].iat[0]
        else:
            joined_df["amount (L)"].iat[i] = 0.0
        
        # should also consider how to compare temperature and humidity given in weather data
        if len(weather_df[weather_df["time"] == time_str]) == 1:
            joined_df["weather"].iat[i] = weather_df[weather_df["time"] == time_str]["weather"].iat[0]
        else :
            joined_df["weather"].iat[i] = weather_df[weather_df["time"] == joined_df["time"].iat[i-1]]["weather"].iat[0]

    x_dfs[zone] = joined_df[["humidity", "temperature", "vis", "uv", "amount (L)", "weather"]]
    y_dfs[zone] = joined_df[["moisture", "ph"]]

C:\Users\kpmea\AppData\Local\Temp\ipykernel_20500\1753497392.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'heavy intensity rain' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  joined_df["weather"].iat[i] = weather_df[weather_df["time"] == time_str]["weather"].iat[0]
C:\Users\kpmea\AppData\Local\Temp\ipykernel_20500\1753497392.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'heavy intensity rain' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  joined_df["weather"].iat[i] = weather_df[weather_df["time"] == time_str]["weather"].iat[0]


In [73]:
x_dfs["A"]

,humidity,temperature,vis,uv,amount (L),weather
0,6.375502,59.271949,92.725291,31.973472,389.485765,heavy intensity rain
1,30.231270,77.365120,30.876055,82.859301,0.000000,heavy intensity rain
2,21.538457,70.814017,44.635027,71.770407,0.000000,broken clouds: 51-84%
3,15.576952,48.037907,53.380940,3.247810,0.000000,broken clouds: 51-84%
4,85.346134,68.605138,55.028911,32.182874,66.365255,fog
...,...,...,...,...,...,...
91,72.716001,49.481515,7.002782,49.575676,0.000000,light shower sleet
92,17.067353,78.070395,95.415362,1.021902,22.027957,smoke
93,33.758002,45.087349,61.859507,50.839277,0.000000,smoke
94,5.386501,66.303104,87.736821,9.983488,0.000000,light rain and snow


In [90]:
# suppose we do some magic and come up with our ideal watering schedule for the next day

watering_schedules = {zone:[random.uniform(0,500) for _ in range(24)] for zone in zones}

In [95]:
# then we want to add scheduled background tasks
# we'll save it in a file

watering_schedules_dict ={"date":datetime.now().strftime('%Y-%m-%d'), "zones":[{"zone":zone, "watering_schedule": watering_schedules[zone]} for zone in zones]}

with open("../database/watering_schedule.json", 'w') as json_file:
    json.dump(watering_schedules_dict, json_file, indent=4)